In [ ]:
#| hide
from fedai.core import *

# fedai

> A minimalist Library for doing Federated learning research.

Fedai enables rapid FL research and prototyping. It's a Library of different federated learning scenarios (iid, non-iid, ...), deep learning models (LLMs, CNNs, MLPs, ...) and datasets that is designed to make federated learning more accessible and accelerate your research. It's also easily extendible so that you can bring your algorithm/data/scneario in a few lines of code. 

If you are new to using `nbdev` here are some useful pointers to get you started.

## Usage

### Installation

Install latest from the GitHub [repository][repo]:

```sh
$ pip install git+https://github.com/Ahmed-Khaled-Saleh/fedai.git
```


[repo]: https://github.com/Ahmed-Khaled-Saleh/fedai

### Documentation

Will be added very soon, but you can read the notebooks since they contain all the necessary information for you to understand how it works.